# System Volt-Var Parameters in IEEE 1547 Format

This notebook illustrates the conversion of system-oriented volt-var function parameters, e.g., slope (gain) and deadband, into the standard table of V1..V4, Q1..Q4 points as defined in IEEE 1547-2018. There are Python code cells in the notebook that create plots and tables to illustrate various points of application and implementation. A live version of this notebook is available at [gridhub](https://github.com/temcdrm/gridhub), under an open-source license and copyright that permit reuse and modification.

Run the following Python code cell to define the plot and table functions using [Matplotlib](https://matplotlib.org/). In this version of *set_characteristic*:

- *Vref* is the center point of the volt-var characteristic, equal to 1 per-unit of the nominal voltage, which is denoted *VN* in the standard. *Vref* must lie in the range 0.95 to 1.05.
- *deadband* must be zero for category A DER, for which V2 = V3 = *Vref*. The maximum *deadband* for category B is 0.06 pu, due to limits on V2 and V3.
- The standard imposes additional constraints, not all of which are checked in this function. For example, the caller could specify non-zero *deadband* for category A, or *slope* higher than allowed by constraints on V1 and V4.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import math

# convert center (Vref), deadband, slope, q limits, and qbias into a table of V and Q points.
# the function returns two arrays for the V and Q points
#    the arrays have sentinel elements below V1 and above V4, so they are 6 elements long (not 4)
#    the sentinel elements clarify that constant extrapolation is used outside the range [V1..V4]
def set_characteristic (Vref=1.0, deadband=0.0, slope=22.0, qmax=0.44, qmin=-0.44, qbias=0.0):
  if Vref < 0.95:  # standard imposes constraints on Vref
    print ('Vref ({:.4f}) is too low, correcting to 0.95'.format (Vref))
    Vref = 0.95
  elif Vref > 1.05:
    print ('Vref ({:.4f}) is too high, correcting to 1.05'.format (Vref))
    Vref = 1.05
  if qbias > qmax: # limit on steady-state reactive power
    print ('Qbias ({:.4f}) is too high, correcting to {:.4f}'.format (qbias, qmax))
    qbias = qmax
  elif qbias < qmin:
    print ('Qbias ({:.4f}) is too low, correcting to {:.4f}'.format (qbias, qmin))
    qbias = qmin
  if deadband < 0.0: # limit on deadband for category B
    print ('deadband ({:.4f}) is too low, correcting to 0.0'.format (deadband))
    deadband = 0.0
  elif deadband > 0.06:
    print ('deadband ({:.4f}) is too high for category B, correcting to 0.06'.format (deadband))
    deadband = 0.06
  Q1 = qmax
  Q2 = qbias
  Q3 = qbias
  Q4 = qmin
  V2 = Vref - 0.5 * deadband
  V3 = Vref + 0.5 * deadband
  V1 = V2 - (Q1 - Q2) / slope
  V4 = V3 - (Q4 - Q3) / slope
  VL = V1 - 0.01
  VH = V4 + 0.01
  vtable = np.array ([VL, V1, V2, V3, V4, VH])
  qtable = np.array ([Q1, Q1, Q2, Q3, Q4, Q4])
  return vtable, qtable

# this function plots and tabulates a volt-var characteristic
def show_characteristic (label, Vref, deadband, slope, qmax, qmin, qbias=0.0):
  vtable, qtable = set_characteristic (Vref, deadband, slope, qmax, qmin, qbias)
  
  # bounds for plotting the horizontal axis
  vmin = vtable[0]-0.01
  vmax = vtable[-1]+0.01
  
  # evaluate the characteristic over 500 equal voltage intervals
  v = np.linspace (vmin, vmax, 501)
  # interpolating Q using the numpy library function
  q = np.interp (v, vtable, qtable)
  
  # create the plot
  fig, ax = plt.subplots(1, 1, figsize=(9,4), tight_layout=True)
  fig.suptitle ('{:s} volt-var characteristic'.format (label))
  
  ax.plot (vtable, qtable, marker='o', color='blue', label='Points and Sentinels')
  ax.plot (v, q, color='red', label='Interpolated')
  ax.grid ()
  ax.set_xlabel ('V [pu]')
  ax.set_ylabel ('Q [pu]')
  ax.set_xlim (vmin, vmax)
  ax.legend ()
  
  # create the data table with 3 columns
  cellText = []
  cellText.append (['INPUTS', '', ''])
  cellText.append (['Vref', '{:.3f}'.format (Vref), ''])
  cellText.append (['deadband', '{:.3f}'.format (deadband), ''])
  cellText.append (['slope', '{:.3f}'.format (slope), ''])
  cellText.append (['Qmax', '{:.3f}'.format (qmax), ''])
  cellText.append (['Qmin', '{:.3f}'.format (qmin), ''])
  cellText.append (['Qbias', '{:.3f}'.format (qbias), ''])
  cellText.append (['', '', ''])
  cellText.append (['TABLE', 'V', 'Q'])
  for i in range(4):
    cellText.append (['{:d}'.format(i+1), '{:.3f}'.format(vtable[i+1]), '{:.3f}'.format(qtable[i+1])])
  cwidth = 0.2
  plt.table (cellText=cellText, cellLoc='center', colWidths=[cwidth, cwidth, cwidth], loc='right')

  plt.show ()

# use the current directory as default location for the "save plot" buttons
# optimize the graphic export for LaTex and PDF
plt.rcParams['savefig.directory'] = os.getcwd()
plt.rcParams['savefig.pad_inches'] = 0.05
plt.rcParams['savefig.dpi'] = 300.0
plt.rcParams['savefig.bbox'] = 'tight'
# invoke the Jupyter support for  Matplotlib graphics
%matplotlib widget


## Examples

Run the following Python code cell to show several volt-var characteristics of interest. The red lines are created from a one-line call to the linear interpolation function in Numpy, using the blue V, Q points for interpolation control. To the right of each graph, the system input parameters and resulting V, Q points are tabulated. As noted in the Python code comments, there are "sentinel" points below V1 and above V4, but these are only used for visual clarity and not included in the table. Constant extrapolation of Q applies below V1 and above V4.

- *Default category A* has no deadband, and a gentle implied slope (gain) of 2.5.
- *Aggressive category A* has the maximum allowed slope for category A, 12.5, based on the ranges of adjustability in the table points. In another notebook, this characteristic will be used with autonomously adjusting reference voltage (AARV) for DER in category A.
- *Default category B* has a deadband with implied slope of 7.333.
- *Aggressive category B* has the maximum allowed slope for category B, 22.0, based on the ranges of adjustability in the table points. The deadband is also set to zero. In another notebook, this characteristic will be used with autonomously adjusting reference voltage (AARV) for DER in category B.
- *Hawaii Rule 14H* has a voltage deadband of 0.06 pu around the center point (*Vref*) of 1.0 pu. This characteristic is designed to mitigate steady-state overvoltage by absorbing reactive power when the voltage is in a range 1.03 - 1.06 pu. Above 1.06 pu, a volt-watt function (not shown) begins to curtail real power output. For voltages below 0.97 pu, the DER will supply reactive power to help mitigate undervoltage.

In [ ]:
show_characteristic ('Default category A',    Vref=1.0, deadband=0.0,  slope=2.5,      qmax=0.25, qmin=-0.25)
show_characteristic ('Aggressive category A', Vref=1.0, deadband=0.0,  slope=12.5,     qmax=0.25, qmin=-0.25)
show_characteristic ('Default category B',    Vref=1.0, deadband=0.04, slope=22.0/3.0, qmax=0.44, qmin=-0.44)
show_characteristic ('Aggressive category B', Vref=1.0, deadband=0.0,  slope=22.0,     qmax=0.44, qmin=-0.44)
show_characteristic ('Hawaii Rule 14H',       Vref=1.0, deadband=0.06, slope=43.0/3.0, qmax=0.44, qmin=-0.44)


## Examples with *Qbias*

Within the scope of IEEE 1547-2018, the table of points may also be shifted with a reactive power bias level, *Qbias*. This has the effect of implementing a constant Q mode, modified by a volt-var response to voltage deviations. This allows DER to participate in steady-state grid voltage control as a reactive power resource, dispatched like shunt capacitors or reactors, but still responding autonomously to local voltage excursions.

Run the following Python code cell to repeat the preceding five example volt-var characteristics, with positive (capacitive) and negative (inductive) bias levels for steady-state reactive power from the DER. In all cases, *Qbias* is 0.1 pu of the DER rating. This bias point is plotted in blue at V=1.00 pu, but it's no longer centered on the horizontal axis.  The intervals V2-V1 and V4-V3 are no longer equal, but they define equal regulation slopes that terminate properly on (V1, Q1) and (V4, Q4). In another workbook, it will be shown that after a voltage fluctuation, the steady-state reactive power will depend on whether AARV has been enabled. With AARV, Q will eventually return to *Qbias* but without AARV, Q will remain at a different value when the voltage remains outside the fixed deadband.

- *Default Category A* has no deadband, so Q is usually not equal to *Qbias*. The red characteristics crosses the horizontal axis at V = 1.0 +/- *Qbias* / *slope* = 1.0 +/- 0.1/2.5 = 1.0 +/- 0.04 pu. Per the standard, when *Vref*=1, V1 must be at least 0.82 pu and V4 no greater than 1.18 pu. The limits are met in this example, but **the limit on V1 or V4 would be violated if the magnitude of *Qbias* exceeds 0.2 pu**.
- *AARV Category A* has no deadband, but with AARV it can regulate Q to *Qbias*. With a higher slope, the red characteristics cross the horizontal axis at 1.0 +/- 0.008 pu.
- *Default Category B* has a deadband, wherein Q would equal *Qbias*. The center of the deadband is at V = 1.0 +/- 0.1/7.333 = 1.0 +/- 0.013636 pu.
- *AARV Category B* has no deadband, but with AARV it can regulate Q to *Qbias*. With a higher slope, the red characteristics cross the horizontal axis at 1.0 +/- 0.004545 pu.
- *HI Rule 14H* has a deadband, wherein Q would equal *Qbias*. The center of the deadband is at V = 1.0 +/- 0.1/14.333 = 1.0 +/- 0.006977 pu.
 

In [ ]:
show_characteristic ('Default Category A, +Qbias', Vref=1.0, deadband=0.0,  slope=2.5, qmax=0.25, qmin=-0.25, qbias=0.1)
show_characteristic ('Default Category A, -Qbias', Vref=1.0, deadband=0.0,  slope=2.5, qmax=0.25, qmin=-0.25, qbias=-0.1)

show_characteristic ('AARV Category A, +Qbias', Vref=1.0, deadband=0.0,  slope=12.5,  qmax=0.25, qmin=-0.25, qbias=0.1)
show_characteristic ('AARV Category A, -Qbias', Vref=1.0, deadband=0.0,  slope=12.5,  qmax=0.25, qmin=-0.25, qbias=-0.1)

show_characteristic ('Default Category B, +Qbias', Vref=1.0, deadband=0.04, slope=22.0/3.0, qmax=0.44, qmin=-0.44, qbias=0.1)
show_characteristic ('Default Category B, -Qbias', Vref=1.0, deadband=0.04, slope=22.0/3.0, qmax=0.44, qmin=-0.44, qbias=-0.1)

show_characteristic ('AARV Category B, +Qbias', Vref=1.0, deadband=0.0,  slope=22.0,  qmax=0.44, qmin=-0.44, qbias=0.1)
show_characteristic ('AARV Category B, -Qbias', Vref=1.0, deadband=0.0,  slope=22.0,  qmax=0.44, qmin=-0.44, qbias=-0.1)

show_characteristic ('HI Rule 14H, +Qbias', Vref=1.0, deadband=0.06, slope=43.0/3.0, qmax=0.44, qmin=-0.44, qbias=0.1)
show_characteristic ('HI Rule 14H, -Qbias', Vref=1.0, deadband=0.06, slope=43.0/3.0, qmax=0.44, qmin=-0.44, qbias=-0.1)

## Examples with Voltage Bias

The previous examples of *Qbias* have shifted the volt-var characteristics up or down on the plots. They may also be shifted left or right on the plots, by specifying *Vref* not equal to 1.0 pu. The result with "voltage bias" is not the same as with "reactive power bias". Furthermore, any voltage bias will be washed out when AARV is enabled. Run the following Python code cell to show some of the differences.

- *Default Category A* with negative voltage bias is functionally the same as with negative reactive power bias. Comparing this graph and table to the first example in the previous section, the (V1,Q1) and (V4,Q4) points match. The interior table points differ, but the red lines match because there is no deadband.
- *Default Category B* with positive voltage bias is **functionally different** from the previous example with positive reactive power bias. The (V1,Q1) and (V4,Q4) points match, but the mis-match of (V2,Q2) and (V3,Q3) is important because of the deadband. 
- *Default Rule 14H* with negative voltage bias is also functionally different from the previous example with negative reactive power bias. There is no DER reactive power at nominal voltage, but during a steady-state overvoltage, the DER absorbs its maximum reactive power before volt-watt engages.

In [ ]:
show_characteristic ('Default Category A, -Vbias', Vref=0.96, deadband=0.0,  slope=2.5, qmax=0.25, qmin=-0.25)
show_characteristic ('Default Category B, +Vbias', Vref=1.013636, deadband=0.04, slope=22.0/3.0, qmax=0.44, qmin=-0.44)
show_characteristic ('HI Rule 14H, -Vbias', Vref=0.993023, deadband=0.06, slope=43.0/3.0, qmax=0.44, qmin=-0.44)

## BSD 3-Clause License

Copyright (c) 2023, Meltran, Inc

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.